In [0]:
%sql
CREATE EXTERNAL LOCATION IF NOT EXISTS `bikeshare-dc-data`
URL 's3://bikeshare-dc-data' WITH (CREDENTIAL `bikeshare-storage-cred`)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS `bikeshare-dc-data` MANAGED LOCATION 's3://bikeshare-dc-data';

In [0]:
%sql
USE CATALOG `bikeshare-dc-data`;
CREATE EXTERNAL VOLUME IF NOT EXISTS bikeshare_schema.raw_landing_zone
LOCATION 's3://bikeshare-dc-data/raw_landing_zone'

In [0]:
%fs ls /Volumes/bikeshare-dc-data/bikeshare_schema/raw_landing_zone/dc_share_data

In [0]:
df =  (spark.readStream.format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", "/Volumes/bikeshare-dc-data/bikeshare_schema/raw_landing_zone/schema")
     .option("cloudFiles.inferColumnTypes", "true")
     .load("/Volumes/bikeshare-dc-data/bikeshare_schema/raw_landing_zone/dc_share_data/")
     )


In [0]:
%sql
DESCRIBE EXTENDED `bikeshare-dc-data`.bronze.dc_rideshare_bt;

In [0]:
# need to make sure the df incoming matches the schema that bronze is expecting
# filter the df down to these rows
"""
ride_id	string	null
rideable_type	string	null
started_at	timestamp	null
ended_at	timestamp	null
start_station_name	string	null
start_station_id	int	null
end_station_name	string	null
end_station_id	int	null
start_lat	double	null
start_lng	double	null
end_lat	double	null
end_lng	double	null
member_casual	string	null
_rescued_data	string	null
"""
df = df.select("ride_id","rideable_type","started_at","ended_at","start_station_name","start_station_id","end_station_name","end_station_id","start_lat","start_lng","end_lat","end_lng","member_casual")

In [0]:
df.count()

In [0]:
%sql
USE CATALOG  `bikeshare-dc-data`;
CREATE SCHEMA IF NOT EXISTS bronze;
  

In [0]:
#  set the Spark SQL configuration spark.sql.files.ignoreMissingFiles to true.
spark.conf.set("spark.sql.files.ignoreMissingFiles", "true")

In [0]:
# need to write the data to a bronze table
writeStream = (df.writeStream
.format("delta")
.option("checkpointLocation", "/Volumes/bikeshare-dc-data/bikeshare_schema/raw_landing_zone/_checkpoint_stream")
.trigger(availableNow=True)
.outputMode("append")
.toTable("`bikeshare-dc-data`.bronze.dc_rideshare_bt"))